In [1]:
import sys
import numpy as np
import pandas as pd

from datatable import dt, f, by

from plotly import express as px, io as pio
pd.options.plotting.backend = 'plotly'
pio.renderers.default = 'plotly_mimetype+notebook_connected'

from utils import get_path, data_load
sys.path.insert(0, '../')
from secret import API_KEY

# Fontes de Dados

## Procedimentos

### Carregando tabela

In [2]:
path_proc = get_path('GESTANTES', 'procs_bienio.csv.gzip')
df_proc_bienio = pd.read_csv(path_proc)
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0.0000,0.000000,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167211,1,1,SL/SC/42016/421870,SL/SC/42007/420540/3157245,138.4773,104.940000,138.4773,104.940000,2,0_1,3_5,-28.4713,-49.0144,-27.5945,-48.5477,1
167212,1,1,SL/SC/42016/421870,SL/SC/42015/420460/2594277,54.6210,49.316667,54.6210,49.316667,2,0_0,3_4,-28.4713,-49.0144,-28.6723,-49.3729,1
167213,1,1,SL/SC/42016/421870,SL/SC/42016/420280/2665883,35.5439,37.493333,71.0878,74.986667,1,0_0,3_3,-28.4713,-49.0144,-28.2681,-49.1701,2
167214,1,1,SL/SC/42016/421870,SL/SC/42016/420730/2385880,53.9610,45.353333,53.9610,45.353333,1,0_0,3_3,-28.4713,-49.0144,-28.2284,-48.6659,1


### Filtrando `UF = RJ`

In [3]:
df_proc = df_proc_bienio[
  (
    df_proc_bienio['origem'].str.contains('/RJ/')
  ) & (
    True # df_proc_bienio['destino'].str.contains('/RJ/')
  )
]
df_proc

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
27697,0,0,SD/RJ/33001/330010,SD/ES/32002/320245/2448203,614.6130,459.768333,1229.2260,919.536667,3,0_0,4_5,-23.0011,-44.3196,-20.2347,-41.5087,2
27698,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010/2280868,0.0000,0.000000,0.0000,0.000000,0,0_0,4_4,-23.0011,-44.3196,-23.0011,-44.3196,533
27699,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010/2281384,0.0000,0.000000,0.0000,0.000000,0,0_0,4_4,-23.0011,-44.3196,-23.0011,-44.3196,116
27700,0,0,SD/RJ/33001/330010,SD/SP/35172/351840/2081512,222.9879,208.666667,222.9879,208.666667,3,0_0,4_4,-23.0011,-44.3196,-22.8075,-45.1938,1
27701,0,0,SD/RJ/33001/330260,SD/RJ/33001/330010/2280868,56.7885,47.100000,56.7885,47.100000,1,0_0,4_4,-22.9594,-44.0409,-23.0011,-44.3196,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156313,1,1,SD/RJ/33009/330590,SD/RJ/33006/330330/0012521,186.0775,184.508333,186.0775,184.508333,2,0_0,3_5,-22.0638,-42.0643,-22.8832,-43.1034,1
156314,1,1,SD/RJ/33009/330590,SD/RJ/33008/330240/5412447,80.5919,75.171667,80.5919,75.171667,2,0_0,3_5,-22.0638,-42.0643,-22.3768,-41.7848,1
156315,1,1,SD/RJ/33009/330590,SD/RJ/33008/330415/2267209,79.7153,75.683333,79.7153,75.683333,2,0_0,3_5,-22.0638,-42.0643,-22.1031,-41.4693,1
156316,1,1,SD/RJ/33009/330590,SD/RJ/33009/330340/2271826,81.5882,79.888333,81.5882,79.888333,1,0_0,3_3,-22.0638,-42.0643,-22.2932,-42.5377,1


### Contabilizando ocorrências

In [4]:
cols_proc = [
  'bienio',
  'parto_normal',
  'count'
]

In [5]:
df_proc = df_proc[cols_proc]
df_proc = df_proc.groupby(cols_proc[:2], as_index=False).sum()
df_proc['bienio'] = df_proc['bienio'].map({
  0: '2010-2011',
  1: '2018-2019',
})
df_proc['parto_normal'] = df_proc['parto_normal'].map({
  0: 'Parto Cesárea',
  1: 'Parto Normal',
})
df_proc = df_proc.rename(columns={
  'bienio': 'Biênio',
  'parto_normal': 'Parto',
  'count': 'Procedimentos',
})
df_proc

,Biênio,Parto,Procedimentos
0,2010-2011,Parto Cesárea,39323
1,2010-2011,Parto Normal,148327
2,2018-2019,Parto Cesárea,82583
3,2018-2019,Parto Normal,148010


# Resultados

## Partos realizados no SUS

Residentes do estado do Rio de Janeiro.

In [6]:
px.bar(
  df_proc,
  x='Parto',
  y='Procedimentos',
  color='Biênio',
  barmode='group',
  title='Número total de partos realizados no SUS (residentes RJ)',
  color_discrete_map={'2010-2011': '#31A885', '2018-2019': '#DD6F1C'},
  text_auto=True
)